# Test connection to adlsg2 using a service principal

In [8]:
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient

# Authenticate using a service principal
credential = ClientSecretCredential(
    tenant_id="***",
    client_id="***",
    client_secret="***"
)

# Create a DataLakeServiceClient to interact with Azure Data Lake Storage Gen2
service_client = DataLakeServiceClient(
    account_url="https://***.dfs.core.windows.net",
    credential=credential
)

# List file systems to test connectivity
filesystems = service_client.list_file_systems()
for fs in filesystems:
    print(fs.name)

taxidata


# Test connection to synapse

In [2]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.synapse import SynapseManagementClient

# Authenticate
credential = DefaultAzureCredential()
subscription_id = '***'
client = SynapseManagementClient(credential, subscription_id)

# List Synapse workspaces
workspaces = client.workspaces.list()
for workspace in workspaces:
    print(workspace.name)

kestrasynapseworkspace
azuresynapsetraining


-----

# Create main monthly table

In [39]:
import pyodbc
from azure.identity import DefaultAzureCredential

# Connection parameters
server = "****.sql.azuresynapse.net"
database = "****"
username = "****"
password = "****"
driver = '{ODBC Driver 17 for SQL Server}'

# Establish connection
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}',
    autocommit=True  # <-- Add this line
)
cursor = conn.cursor()

create_table_sql = """
IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'green_tripdata')
CREATE TABLE green_tripdata (
    unique_row_id VARCHAR(255) NOT NULL,
    filename VARCHAR(255) NOT NULL,
    VendorID VARCHAR(255),
    lpep_pickup_datetime DATETIME2,
    lpep_dropoff_datetime DATETIME2,
    store_and_fwd_flag VARCHAR(255),
    RatecodeID VARCHAR(255),
    PULocationID VARCHAR(255),
    DOLocationID VARCHAR(255),
    passenger_count INT,
    trip_distance DECIMAL(18, 2),
    fare_amount DECIMAL(18, 2),
    extra DECIMAL(18, 2),
    mta_tax DECIMAL(18, 2),
    tip_amount DECIMAL(18, 2),
    tolls_amount DECIMAL(18, 2),
    ehail_fee DECIMAL(18, 2),
    improvement_surcharge DECIMAL(18, 2),
    total_amount DECIMAL(18, 2),
    payment_type INT,
    trip_type VARCHAR(255),
    congestion_surcharge DECIMAL(18, 2)
  )
WITH(
      DISTRIBUTION = HASH(unique_row_id),
      CLUSTERED COLUMNSTORE INDEX
      );
"""

cursor.execute(create_table_sql)
cursor.close()
conn.close()


# Create External Table

In [17]:
import pyodbc

# Connection parameters
server = "****.sql.azuresynapse.net"
database = "****"
username = "****"
password = "****"
driver = '{ODBC Driver 17 for SQL Server}'

# Establish connection with autocommit=True
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}',
    autocommit=True
)
cursor = conn.cursor()


# Step 1: Create database-scoped credential
credential_sql = """
IF NOT EXISTS (SELECT * FROM sys.database_scoped_credentials WHERE name = 'WorkspaceIdentity')
CREATE DATABASE SCOPED CREDENTIAL WorkspaceIdentity
WITH
    IDENTITY = 'Managed Identity',
    SECRET = '';
"""
cursor.execute(credential_sql)
# cursor.close()
# conn.close()

# Step 2: Create external data source
data_source_sql = """
IF NOT EXISTS (SELECT * FROM sys.external_data_sources WHERE name = 'ADLSGen2')
CREATE EXTERNAL DATA SOURCE ADLSGen2
WITH (
    LOCATION = 'abfss://taxidata@kestradatalake.dfs.core.windows.net',
    CREDENTIAL = WorkspaceIdentity  -- Reference the credential here
);
"""
cursor.execute(data_source_sql)

# Step 3: Create file format
file_format_sql = """
IF NOT EXISTS (SELECT * FROM sys.external_file_formats WHERE name = 'CSVFormat')
CREATE EXTERNAL FILE FORMAT CSVFormat 
WITH (
    FORMAT_TYPE = DELIMITEDTEXT,
    FORMAT_OPTIONS (
        FIELD_TERMINATOR = ',',
        FIRST_ROW = 2
    )
);
"""
cursor.execute(file_format_sql)

# Step 4: Create external table
create_external_sql = """
IF NOT EXISTS (SELECT * FROM sys.external_tables WHERE name = 'green_tripdata_ext')
CREATE EXTERNAL TABLE green_tripdata_ext (
    VendorID VARCHAR(255),
    lpep_pickup_datetime DATETIME2,
    lpep_dropoff_datetime DATETIME2,
    store_and_fwd_flag VARCHAR(255),
    RatecodeID VARCHAR(255),
    PULocationID VARCHAR(255),
    DOLocationID VARCHAR(255),
    passenger_count INT,
    trip_distance DECIMAL(18, 2),
    fare_amount DECIMAL(18, 2),
    extra DECIMAL(18, 2),
    mta_tax DECIMAL(18, 2),
    tip_amount DECIMAL(18, 2),
    tolls_amount DECIMAL(18, 2),
    ehail_fee DECIMAL(18, 2),
    improvement_surcharge DECIMAL(18, 2),
    total_amount DECIMAL(18, 2),
    payment_type INT,
    trip_type VARCHAR(255),
    congestion_surcharge DECIMAL(18, 2)
)
WITH (
    LOCATION = 'green_taxi/green_tripdata_2019-01.csv',
    DATA_SOURCE = ADLSGen2,
    FILE_FORMAT = CSVFormat
);
"""
cursor.execute(create_external_sql)

cursor.close()
conn.close()

# Merge Into main table 

In [ ]:
import pyodbc
from azure.identity import DefaultAzureCredential

# Connection parameters
server = "****.sql.azuresynapse.net"
database = "****"
username = "****"
password = "****"
driver = '{ODBC Driver 17 for SQL Server}'

# Establish connection with autocommit=True
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}',
    autocommit=True
)
cursor = conn.cursor()

merge_sql = """
          MERGE INTO green_tripdata AS Target
          USING (
              SELECT 
                  CAST(HASHBYTES('MD5', 
                      CONCAT(
                          COALESCE(VendorID, ''),
                          COALESCE(CAST(lpep_pickup_datetime AS NVARCHAR(255)), ''),
                          COALESCE(CAST(lpep_dropoff_datetime AS NVARCHAR(255)), ''),
                          COALESCE(PULocationID, ''),
                          COALESCE(DOLocationID, '')
                      ) 
                      ) AS VARCHAR(255)
                  ) AS unique_row_id,
                  'green_tripdata_2019-01.csv' AS filename,
                  *
              FROM green_tripdata_ext
          ) AS Source
          ON Target.unique_row_id = Source.unique_row_id
          WHEN NOT MATCHED THEN
              INSERT (unique_row_id, filename, VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
)
              VALUES (Source.unique_row_id, Source.filename, source.VendorID, source.lpep_pickup_datetime, source.lpep_dropoff_datetime, source.store_and_fwd_flag, source.RatecodeID, source.PULocationID, source.DOLocationID, source.passenger_count, source.trip_distance, source.fare_amount, source.extra, source.mta_tax, source.tip_amount, source.tolls_amount, source.ehail_fee, source.improvement_surcharge, source.total_amount, source.payment_type, source.trip_type, source.congestion_surcharge);
          """
cursor.execute(merge_sql)

truncate_sql = "TRUNCATE TABLE green_tripdata_ext"
cursor.execute(truncate_sql)

cursor.close()
conn.close()


# Modify database objects

In [10]:
import pyodbc
from azure.identity import DefaultAzureCredential

# Connection parameters
server = "****.sql.azuresynapse.net"
database = "****"
username = "****"
password = "****"
driver = '{ODBC Driver 17 for SQL Server}'

# Establish connection with autocommit=True
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}',
    autocommit=True
)
cursor = conn.cursor()


alter_unique_id_data_type_sql = "DROP TABLE yellow_tripdata"

cursor.execute(alter_unique_id_data_type_sql)